In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)

### CausalConv1d

- 시계열, Autoregressive model에 사용
- 왼쪽(과거)에 패딩, 오른쪽(미래)에는 패딩 없이
- 1D Conv이므로 weight.shape = (32, 16, 3)  [in_channels=16, out_channels=32, kernel_size=3]

In [ ]:
class CausalConv1d(nn.Conv1d):
    """
    홀수 커널
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 왼쪽 부분 얼만큼 패딩 줄지
        self.causal_padding = self.dilation[0] * (self.kernel_size[0] - 1)

    def forward(self, x):
        # 왼쪽으로 self.causal_padding만큼 0-padding
        # 오른쪽 패딩은 주지 않음([self.causal_padding, 0])
        return self._conv_forward(
            F.pad(x, [self.causal_padding, 0]), self.weight, self.bias
        )

In [8]:
x = torch.arange(8, dtype=torch.float32).reshape(1, 1, -1)
print("Input shape:", x.shape)
print("Input tensor:", x)

causal_conv = CausalConv1d(in_channels=1, out_channels=1,
                           kernel_size=3, dilation=2, bias=False)

y = causal_conv(x)

print("Output shape:", y.shape)
print("Output tensor:", y)

Input shape: torch.Size([1, 1, 8])
Input tensor: tensor([[[0., 1., 2., 3., 4., 5., 6., 7.]]])
Output shape: torch.Size([1, 1, 8])
Output tensor: tensor([[[ 0.0000, -0.4752, -0.9504, -1.1158, -1.2813, -1.4511, -1.6209,
          -1.7907]]], grad_fn=<ConvolutionBackward0>)


In [ ]:
# 짝수 커널 처리 함수형
def causal_padding(x, kernel):
    if kernel.shape[-1] % 2 == 0:
        kernel = F.pad(kernel, [1,0], value=0.0)

    x = F.pad(x, [kernel.shape[-1]-1, 0], value=0.0)
    return x, kernel


def causal_conv(x, kernel, bias=None, **kwargs):
    x, kernel = causal_padding(x, kernel)
    return F.conv1d(x, kernel, bias=bias, padding=0)


In [ ]:
class CausalConv1d(nn.Conv1d):
    """
    짝수 커널도 처리
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def forward(self, x):
        # (1) weight를 가져옴
        weight = self.weight  # shape: (out_ch, in_ch, K)

        # (2) 커널 길이가 짝수라면 왼쪽 1칸을 pad해
        #     실제 연산 시에는 "홀수 커널"이 되도록 만듦
        # 예: kernel_size=4 → shape=(out_ch, in_ch, 5)
        if weight.shape[-1] % 2 == 0:
            weight = F.pad(weight, [1, 0], value=0.0)

        # (3) 최종 커널 크기(홀수)
        kernel_size = weight.shape[-1]

        # (4) 인과적 패딩(왼쪽) 계산: dilation*(kernel_size - 1)
        #     → 미래 정보가 들어오지 않도록 왼쪽만 패딩
        causal_padding = self.dilation[0] * (kernel_size - 1)

        # (5) 입력 x에 왼쪽만 0-패딩
        x = F.pad(x, [causal_padding, 0], value=0.0)

        # (6) conv1d 수행 (padding=0)
        return F.conv1d(x, weight, self.bias,
                         self.stride, 0,
                         self.dilation, self.groups)

In [ ]:
# (배치=1, 채널=1, 길이=10) 형태의 입력을 생성 (0~9 범위)
x = torch.arange(10, dtype=torch.float32).reshape(1, 1, -1)
print("Input x shape:", x.shape)
print("Input x:", x)

# 커널 사이즈=4(짝수)로 설정 → forward에서 자동으로 한 칸 pad하여 홀수화
causal_conv = CausalConv1d(in_channels=1, out_channels=1, kernel_size=4, bias=False)

# Forward
y = causal_conv(x)
print("Output y shape:", y.shape)
print("Output y:", y)

### CausalConvTranspose1d
- ConvTranspose인데 causal 버전
- 미래 시점을 보지 않도록 함
- 디코딩, 업샘플에서 causal conv에 반대되는 개념으로 쓸 수 있음

In [ ]:
class CausalConvTranspose1d(nn.ConvTranspose1d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 뒤쪽(미래)을 일정 길이만큼 제거
        self.causal_padding = (
            self.dilation[0] * (self.kernel_size[0] - 1)
            + self.output_padding[0]
            + 1
            - self.stride[0]
        )
    
    def forward(self, x, output_size=None):
        if self.padding_mode != 'zeros':
            raise ValueError('Only `zeros` padding mode is supported for ConvTranspose1d')

        assert isinstance(self.padding, tuple)
        output_padding = self._output_padding(
            x, output_size, self.stride, self.padding, self.kernel_size, self.dilation)

        # 실제 Transposed Conv 진행
        out = F.conv_transpose1d(
            x, self.weight, self.bias, self.stride, self.padding,
            output_padding, self.groups, self.dilation
        )
        # 인과적 구조를 위해 끝부분(미래) 잘라내기
        return out[..., :-self.causal_padding]

### Causal Conv Encoder+Decoder

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dilation):
        super().__init__()
        self.dilation = dilation
        self.layers = nn.Sequential(
            CausalConv1d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=7,
                dilation=dilation
            ),
            nn.ELU(),
            nn.Conv1d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=1
            )
        )

    def forward(self, x):
        return x + self.layers(x)

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, out_channels, stride):
        super().__init__()
        self.layers = nn.Sequential(
            CausalConvTranspose1d(
                in_channels=2*out_channels,
                out_channels=out_channels,
                kernel_size=2*stride, 
                stride=stride
            ),
            nn.ELU(),
            ResBlock(
                in_channels=out_channels, 
                out_channels=out_channels,
                dilation=1
            ),
            nn.ELU(),
            ResBlock(
                in_channels=out_channels, 
                out_channels=out_channels,
                dilation=3
            ),
            nn.ELU(),
            ResBlock(in_channels=out_channels, out_channels=out_channels,
                         dilation=9),
        )

### Inverted separable convolution

In [ ]:
class SepConv(nn.Module):
    """
    Inverted separable convolution from MobileNetV2: https://arxiv.org/abs/1801.04381.
    """
    def __init__(self, dim, expansion_ratio=2, act1_layer=StarReLU, act2_layer=nn.Identity,
                 bias=False, kernel_size=7, padding=3, **kwargs,):
        super().__init__()
        med_channels = int(expansion_ratio * dim)
        self.pwconv1 = nn.Linear(dim, med_channels, bias=bias)
        self.act1 = act1_layer()
        self.dwconv = nn.Conv2d(
            med_channels, med_channels, kernel_size=kernel_size,
            padding=padding, groups=med_channels, bias=bias
        )
        self.act2 = act2_layer()
        self.pwconv2 = nn.Linear(med_channels, dim, bias=bias)

    def forward(self, x):
        x = self.pwconv1(x)
        x = self.act1(x)
        # [B, H, W, C] => [B, C, H, W]
        x = x.permute(0, 3, 2, 1)
        x = self.dwconv(x)
        # [B, C, H, W] => [B, H, W, C]
        x = x.permute(0, 2, 3, 1)
        x = self.act2(x)
        x = self.pwconv2(x)
        return x

In [23]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):

        # args와 kwargs에서 파라미터 이름을 얻기 위해 사용
        param_names = func.__code__.co_varnames[1:func.__code__.co_argcount]
        params = dict(zip(param_names, args))  # 위치인자 처리
        print(params)
        params.update(kwargs)  # 키워드 인자 업데이트

        for key, value in params.items():
            setattr(self, key, value)

        func(self, *args, **kwargs)  # 먼저 원래의 초기화 함수를 실행합니다.
    return wrapper

class SimpleLoss(nn.Module):
    @my_decorator
    def __init__(self, domain_rgb, l1_lambda, l2_lambda):
        super().__init__()
        # 추가적인 초기화 코드가 여기에 포함될 수 있습니다.
        self.l1_loss = nn.L1Loss() if self.l1_lambda != 0 else None
        self.l2_loss = nn.MSELoss() if self.l2_lambda != 0 else None

# 테스트
SimpleLoss(1, 2, 3)

{'domain_rgb': 1, 'l1_lambda': 2, 'l2_lambda': 3}


SimpleLoss(
  (l1_loss): L1Loss()
  (l2_loss): MSELoss()
)